Queremos hacer un estudio de mercado del sector del automóvil en España, cómo han evolucionado las ventas a lo largo del tiempo.

de aquí saco las matriculaciones por año: https://datosmacro.expansion.com/negocios/matriculaciones-vehiculos/espana?anio=2021

de aquí saco una API con los modelos = https://public.opendatasoft.com/explore/dataset/all-vehicles-model/api/

API para modelos por años: https://rapidapi.com/koukoulina001/api/cars21/playground/apiendpoint_23fe4099-3684-4e75-8a43-9c7a880e34ba

Aquí info sobre modelos: https://anfac.com/cifras-clave/matriculaciones-turismos-y-todoterreno/

Coches más vendidos por años: https://www.autofacil.es/industria/coches-mas-vendidos-historia-espana/178791.html

API Wallapop: http://api.wallapop.com/api/v3/general/search


In [2]:
import requests
import pandas as pd

In [ ]:
#Test API wallapop: FUNCIONA!!!!!!!!

url = "http://api.wallapop.com/api/v3/general/search"

headers = {'Accept': '*/*', 'User-Agent': 'Wget/1.21.4',

'Accept-Encoding': 'identity', 'X-DeviceOS': '0' }

# Latitud: . Longitud: 

params = {"search_objects":[],
          "from":0,
          "to":100,
        #   "distance_ordered":false,
        #   "experiment":null,
        #   "experiment_other_properties":null,
          "keywords":"Iphone",
          "min_sale_price" : 0,
          "max_sale_price" : 200,
          "order":"most_relevance",
        #   "bubble":null,
          "search_point":{"latitude":40.450381,"longitude":-3.518064},
        #   "spellcheck":null
        }

res = requests.get(url, headers = headers, params = params)
res.status_code

dc_wallapop = res.json()
